In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import sys
import json
import os 

In [3]:
path_groundtruth="E:\\uwb_verileri\\processe\\uwb_data\\data_23_04\\data_8.txt\\kf_lms.txt"
path_anchors="E:\\uwb_verileri\\processe\\uwb_data\\data_23_04\\data_8.txt\\8_SID_6_raw.txt"

In [4]:
filename = os.path.basename(os.path.dirname(path_groundtruth))[0:7]

In [5]:
class ExtractDataFromTxt():
    def __init__(self):
        try:
            pass
        except Exception as e:
            raise ExtractDataFromTxt(e,sys)
    
    def txt_to_df_groundtruth_converter(self,path_groundtruth,path_anchors,save=True):
        gt_data = []
        filename = os.path.basename(os.path.dirname(path_groundtruth))[0:7]
        
        with open(path_groundtruth, "r") as file:
            for line in file:
                json_line = line.strip().replace("'", "\"")
                gt_data.append(json.loads(json_line))
        df_gt = pd.DataFrame(gt_data)
        pivot_df_positions = df_gt.pivot_table(index="sequence_number", values=["position"],aggfunc=lambda x: list(x))
        desired_sequence_numbers =df_gt["sequence_number"].unique()

        uwb_datas=[]
        with    open (path_anchors,"r") as file :
            for line in file :
                uwb_data=json.loads(line.strip())
                uwb_datas.append(uwb_data)
            uwb_df=pd.DataFrame(uwb_datas)
        filtered_uwb_df = uwb_df[uwb_df['sequence_number'].isin(desired_sequence_numbers)]
        pivot_df_uwb = filtered_uwb_df.pivot_table(index="sequence_number", columns="anchor_id", values="distance")
        merged_df = pd.merge(pivot_df_uwb, pivot_df_positions, left_index=True, right_index=True, how='left')
        for column in merged_df.select_dtypes(include=[np.number]).columns:   
            merged_df[column] = merged_df[column].fillna(merged_df[column].rolling(window=5, min_periods=1).mean())
            merged_df.fillna(method='ffill', inplace=True)
            merged_df.fillna(method='bfill', inplace=True)
            merged_df['position_x'] = merged_df['position'].apply(lambda x: x[0][0])  # X koordinatı
            merged_df['position_y'] = merged_df['position'].apply(lambda x: x[0][1]) 
            if save==True:
                merged_df.to_csv(f"{filename}_merged.csv", index=True)
                merged_df.to_pickle(f"{filename}_merged.pkl")
            else:
                pass
        return merged_df

        

In [6]:
bakalim=ExtractDataFromTxt()

In [7]:
merged_df=bakalim.txt_to_df_groundtruth_converter(path_groundtruth,path_anchors,save=False)

In [8]:
merged_df

,27,104,133,134,149,199,position,position_x,position_y
sequence_number,,,,,,,,,
7437,10.2060,16.514,27.668,28.824,14.092,3.898,"[[9.8, 1.14]]",9.80,1.14
7438,10.1760,16.482,27.668,28.788,14.152,3.876,"[[9.8, 0.93]]",9.80,0.93
7439,10.1960,16.520,27.668,28.846,14.110,3.894,"[[9.84, 0.83]]",9.84,0.83
7441,10.1960,16.518,27.668,28.808,14.082,3.956,"[[9.86, 0.85]]",9.86,0.85
7453,10.3500,16.554,27.668,28.808,14.082,3.920,"[[10.09, 0.95]]",10.09,0.95
...,...,...,...,...,...,...,...,...,...
7593,11.6205,8.862,24.668,22.258,13.258,11.268,"[[2.12, 4.07]]",2.12,4.07
7596,10.0140,8.862,28.860,22.258,16.540,9.784,"[[4.34, 1.02]]",4.34,1.02
7597,4.0420,8.862,28.860,22.258,17.388,9.878,"[[4.01, -1.71]]",4.01,-1.71
